# Let's Keep it Simple

# Text Classification with BERT

In [ ]:
# Kaggle API for downloading dataset
# downloading kaggle.json file from google drive
# important do not share this as file contains downloadable link
import gdown
file = 'shareable link '
output = 'kaggle.json'
gdown.download(file, output, quiet=False) 

# Let's make sure the kaggle.json file is present.
!ls -lha kaggle.json

# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle # uncomment them and run if restarted the kernel
!cp kaggle.json ~/.kaggle/ 

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

# Copy the dataset locally
#!kaggle competitions download titanic -f train.csv


In [1]:
# importing usefull libraries
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd

# We're working on [IMDB reviews dataset](http://nlpprogress.com/english/sentiment_analysis.html) 

# We are going to use the state-of-the-art model BERT to classify between positive and negative reviews.

![picture](https://storage.googleapis.com/kaggle-datasets-images/134715/320111/9230bbbadb805f8018c4dc289b72dd8c/data-original.png?t=2019-05-24-07-03-05)

## Loading the dataset

In [2]:
(ds_train, ds_test), ds_info = tfds.load('imdb_reviews', 
                                         split=(tfds.Split.TRAIN, tfds.Split.TEST), 
                                         as_supervised=True, 
                                         with_info=True)
print('Info about the dataset --> ', ds_info)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteSB55AJ/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteSB55AJ/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteSB55AJ/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.
Info about the dataset -->  tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
   

When using transformers library we first load the tokenizer for the model we would like to use. Then we will proceed as follows:


In [15]:
# installing transformers library
!pip install transformers

     |████████████████████████████████| 1.3MB 4.7MB/s 
     |████████████████████████████████| 1.1MB 13.5MB/s 
     |████████████████████████████████| 890kB 31.1MB/s 
     |████████████████████████████████| 2.9MB 41.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=2370d1f0dab82a951c1e02f79714396d2a8ce86e3fbc1e62a39fcab9667c5c6e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [17]:
# checking the installation of transformers
# if it classifies the statement you have passed then "transformers" is properly installed
!python -c "from transformers import pipeline; print(pipeline('sentiment-analysis')('we love you'))"

2020-11-08 12:51:44.353501: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Downloading: 100% 629/629 [00:00<00:00, 742kB/s]
Downloading: 100% 268M/268M [00:04<00:00, 59.0MB/s]
Downloading: 100% 232k/232k [00:00<00:00, 886kB/s]
Downloading: 100% 230/230 [00:00<00:00, 208kB/s]
[{'label': 'POSITIVE', 'score': 0.9998704791069031}]


In [9]:
for review, label in tfds.as_numpy(ds_train.take(5)):
    print('review', review.decode()[0:50], label)

review This was an absolutely terrible movie. Don't be lu 0
review I have been known to fall asleep during films, but 0
review Mann photographs the Alberta Rocky Mountains in a  0
review This is the kind of film for a snowy Sunday aftern 1
review As others have mentioned, all the women that go nu 1


## Tokenizer
Now we need to apply BERT tokenizer to use pre-trained tokenizers, see HuggingFace Tokenizers. The tokenizers should also match the core model that we would like to use as the pre-trained, e.g. cased and uncased version. The code for loading the tokenizer is as simple as:

In [22]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

The BERT tokenizer uses WordPiece vocabulary. It has over 30000 words and it maps pre-trained embeddings for each. Each word has its ids, we would need to map the tokens to those ids.

In [23]:

vocabulary = tokenizer.get_vocab()

print(list(vocabulary.keys())[5000:5020])

['knight', 'lap', 'survey', 'ma', '##ow', 'noise', 'billy', '##ium', 'shooting', 'guide', 'bedroom', 'priest', 'resistance', 'motor', 'homes', 'sounded', 'giant', '##mer', '150', 'scenes']


In [24]:
max_length_test = 20
test_sentence = 'Test tokenization sentence. Followed by another sentence'

# add special tokens

test_sentence_with_special_tokens = '[CLS]' + test_sentence + '[SEP]'

tokenized = tokenizer.tokenize(test_sentence_with_special_tokens)

print('tokenized', tokenized)

# convert tokens to ids in WordPiece
input_ids = tokenizer.convert_tokens_to_ids(tokenized)
  
# precalculation of pad length, so that we can reuse it later on
padding_length = max_length_test - len(input_ids)

# map tokens to WordPiece dictionary and add pad token for those text shorter than our max length
input_ids = input_ids + ([0] * padding_length)

# attention should focus just on sequence with non padded tokens
attention_mask = [1] * len(input_ids)

# do not focus attention on padded tokens
attention_mask = attention_mask + ([0] * padding_length)

# token types, needed for example for question answering, for our purpose we will just set 0 as we have just one sequence
token_type_ids = [0] * max_length_test

bert_input = {
    "token_ids": input_ids,
    "token_type_ids": token_type_ids,
    "attention_mask": attention_mask
}
print(bert_input)

tokenized ['[CLS]', 'test', 'token', '##ization', 'sentence', '.', 'followed', 'by', 'another', 'sentence', '[SEP]']
{'token_ids': [101, 3231, 19204, 3989, 6251, 1012, 2628, 2011, 2178, 6251, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


These methods `tokenize` and `convert_token_to_ids` can be simplified into just one method called encode_plus, which will also add special tokens like `[CLS]` and `[SEP]` for us.

`[CLS]` at the start of the sentence

`[SEP]` at the end of the sentence

In [25]:

bert_input = tokenizer.encode_plus(
                        test_sentence,                      
                        add_special_tokens = True, # add [CLS], [SEP]
                        max_length = max_length_test, # max length of the text that can go to BERT
                        pad_to_max_length = True, # add [PAD] tokens
                        return_attention_mask = True, # add attention mask to not focus on pad tokens
              )

print('encoded', bert_input)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


encoded {'input_ids': [101, 3231, 19204, 3989, 6251, 1012, 2628, 2011, 2178, 6251, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1944: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


This is taken from [HuggingFace glossary](https://huggingface.co/transformers/glossary.html):

**Input IDs** - The input ids are often the only required parameters to be passed to the model as input. They are token indices, numerical representations of tokens building the sequences that will be used as input by the model.

**Attention mask** - Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]: 1 for tokens that are NOT MASKED, 0 for MASKED tokens.

**Token type ids** - Some models’ purpose is to do sequence classification or question answering. These require two different sequences to be encoded in the same input IDs. They are usually separated by special tokens, such as the classifier and separator tokens. For example, the BERT model builds its two sequence input as such:

Before we can use such input for BERT fine-tunning, we need to be first aware that BERT is trained to consume sequence with maximum of 512 tokens. Let's define our **batch size** and **max length** allowed for our reviews.

## Encoding and Training the dataset

In [26]:

# can be up to 512 for BERT
max_length = 512
batch_size = 6


Now let's combine the whole encoding process to one function so that we can map over our train and test dataset.



In [27]:
def convert_example_to_feature(review):
  
  # combine step for tokenization, WordPiece vector mapping, adding special tokens as well as truncating reviews longer than the max length
  
  return tokenizer.encode_plus(review, 
                add_special_tokens = True, # add [CLS], [SEP]
                max_length = max_length, # max length of the text that can go to BERT
                pad_to_max_length = True, # add [PAD] tokens
                return_attention_mask = True, # add attention mask to not focus on pad tokens
              )


When we will now iterate over again we can apply the encode function for each item.

In [28]:
# map to the expected input to TFBertForSequenceClassification, see here 
def map_example_to_dict(input_ids, attention_masks, token_type_ids, label):
  return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }, label

def encode_examples(ds, limit=-1):

  # prepare list, so that we can build up final TensorFlow dataset from slices.
  input_ids_list = []
  token_type_ids_list = []
  attention_mask_list = []
  label_list = []

  if (limit > 0):
      ds = ds.take(limit)
    
  for review, label in tfds.as_numpy(ds):

    bert_input = convert_example_to_feature(review.decode())
  
    input_ids_list.append(bert_input['input_ids'])
    token_type_ids_list.append(bert_input['token_type_ids'])
    attention_mask_list.append(bert_input['attention_mask'])
    label_list.append([label])

  return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, label_list)).map(map_example_to_dict)


Now use our encode_examples function to convert our train and test datasets.

In [29]:

# train dataset
ds_train_encoded = encode_examples(ds_train).shuffle(10000).batch(batch_size)

# test dataset
ds_test_encoded = encode_examples(ds_test).batch(batch_size)


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1944: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


## Model Initialization

We will use already prepared TensorFlow models from transformers models. You can just import them from the library and call `from_pretrained` and you will be able to use them

In [30]:
from transformers import TFBertForSequenceClassification

# recommended learning rate for Adam 5e-5, 3e-5, 2e-5
learning_rate = 2e-5

# we will do just 1 epoch for illustration, though multiple epochs might be better as long as we will not overfit the model
number_of_epochs = 1


# model initialization
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

# choosing Adam optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)

# we do not have one-hot vectors, we can use sparce categorical cross entropy and accuracy
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertForSequenceClassification: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier', 'dropout_37']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Fine Tuning

### Training

Now we can start fine-tuning process. We will again use the Keras API model.fit and just pass the model configuration, that we have already defined.

In [31]:
bert_history = model.fit(ds_train_encoded, epochs=number_of_epochs, validation_data=ds_test_encoded)

4167/4167 [==============================] - 2573s 617ms/step - loss: 0.2409 - accuracy: 0.9045 - val_loss: 0.1789 - val_accuracy: 0.9319


If you are getting the Resource error, you would need to run this on the GPU/TPU with higher VRAM. At least 12GB is recommended. You might consider to decrease input size/batch size or train in half precision.

### Evaluation



With our simple fine-tuning process, we have already achieved over 93% on the test dataset. We likely underfit, however there are ways to improve further, e.g. larger BERT model, regularization, more epochs, further in task pretraining. 



```
4167/4167 [==============================] - 2573s 617ms/step
- loss: 0.2409 - accuracy: 0.9045
- val_loss: 0.1789 - val_accuracy: 0.9319

```



This is quite good for the initial try in comparison with [current state of the art](https://nlpprogress.com/english/sentiment_analysis.html). 

### Tips for Fine-Tuning

* Use larger BERT model
* Further in-task pretraining
* Multi-task fine-tuning has lower effect than further pre-training, but can help as well, see (Sun et al. 2019)
* Do not constrain yourself just to BERT. Other transformers such as XLNet can work even better.


### References

* [Stanford CS224N: NLP with Deep Learning](https://https://www.youtube.com/watch?v=8rXD5-xhemo)

* [Stanford CS224U Natural Language Understanding](https://www.youtube.com/watch?v=tZ_Jrc_nRJY&list=PLoROMvodv4rObpMCir6rNNUlFAn56Js20)

* [HuggingFace transformers library](https://huggingface.co/)

* [BERT, ULMFit, ELMO](https://jalammar.github.io/illustrated-bert/)

* [BERTViz - Vizualization of Attention Heads](https://github.com/jessevig/bertviz)

* [Illustrated transformer](https://jalammar.github.io/illustrated-transformer/)

* [Illustrated GPT-2](https://jalammar.github.io/illustrated-gpt2)

* http://nlpprogress.com/

* [GLUE benchmark ](https://gluebenchmark.com/tasks)

* Jacob Devlin, Ming-Wei Chang, Kenton Lee, and Kristina Toutanova, BERT:  Pre-training of deep bidirectional transformers for languageunderstanding, 2018.

* Scott Gray, Alec Radford, and Diederik P. Kingma, GPU kernels for block-sparse weights, 2017.

* Jeremy Howard and Sebastian Ruder, Universal language model fine-tuning for text classification, 2018.

* Sepp Hochreiter and Jurgen Schmidhuber, Long short-term memory, Neural computation (1997), no. 8, 1735–1780.

* Alec Radford, Jeff Wu, Rewon Child, David Luan, Dario Amodei, and Ilya Sutskever, Language models are unsupervised multitask learners.

* Chi Sun, Xipeng Qiu, Yige Xu, and Xuanjing Huang, How to fine-tune BERT for text classification?, 2019.

* Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, LlionJones, Aidan N. Gomez, Lukasz Kaiser, and Illia Polosukhin, Attention is all you need, 2017.

* Alex Wang, Amanpreet Singh, Julian Michael, Felix Hill, Omer Levy,and Samuel R. Bowman,Glue:  A multi-task benchmark and analysisplatform for natural language understanding, 2018.


* Zhilin Yang, Zihang Dai, Yiming Yang, Jaime Carbonell, RuslanSalakhutdinov, and Quoc V. Le,Xlnet:  Generalized autoregressive pretraining for language understanding, 2019


